In [1]:
from gurobipy import * 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import copy
import geopandas as gpd 
%matplotlib inline 

In [2]:
products = ["Prod1", "Prod2", "Prod3", "Prod4", "Prod5", "Prod6", "Prod7"]
machines = ["grinder", "vertDrill", "horiDrill", "borer", "planer"]
time_periods = ["January", "February", "March", "April", "May", "June"]

In [3]:
profit_contribution = {"Prod1":10, "Prod2":6, "Prod3":8, "Prod4":4, "Prod5":11, "Prod6":9, "Prod7":3}

In [4]:
time_table = {
    "grinder": {    "Prod1": 0.5, "Prod2": 0.7, "Prod5": 0.3,
                    "Prod6": 0.2, "Prod7": 0.5 },
    "vertDrill": {  "Prod1": 0.1, "Prod2": 0.2, "Prod4": 0.3,
                    "Prod6": 0.6 },
    "horiDrill": {  "Prod1": 0.2, "Prod3": 0.8, "Prod7": 0.6 },
    "borer": {      "Prod1": 0.05,"Prod2": 0.03,"Prod4": 0.07,
                    "Prod5": 0.1, "Prod7": 0.08 },
    "planer": {     "Prod3": 0.01,"Prod5": 0.05,"Prod7": 0.05 }
}

In [5]:
# number of machines down
down = {("January","grinder"): 1, ("February", "horiDrill"): 2, ("March", "borer"): 1,
        ("April", "vertDrill"): 1, ("May", "grinder"): 1, ("May", "vertDrill"): 1,
        ("June", "planer"): 1, ("June", "horiDrill"): 1}

In [6]:
# number of each machine available
qMachine = {"grinder":4, "vertDrill":2, "horiDrill":3, "borer":1, "planer":1} 

In [7]:
# market limitation of sells
upper = {
    ("January", "Prod1") : 500,
    ("January", "Prod2") : 1000,
    ("January", "Prod3") : 300,
    ("January", "Prod4") : 300,
    ("January", "Prod5") : 800,
    ("January", "Prod6") : 200,
    ("January", "Prod7") : 100,
    ("February", "Prod1") : 600,
    ("February", "Prod2") : 500,
    ("February", "Prod3") : 200,
    ("February", "Prod4") : 0,
    ("February", "Prod5") : 400,
    ("February", "Prod6") : 300,
    ("February", "Prod7") : 150,
    ("March", "Prod1") : 300,
    ("March", "Prod2") : 600,
    ("March", "Prod3") : 0,
    ("March", "Prod4") : 0,
    ("March", "Prod5") : 500,
    ("March", "Prod6") : 400,
    ("March", "Prod7") : 100,
    ("April", "Prod1") : 200,
    ("April", "Prod2") : 300,
    ("April", "Prod3") : 400,
    ("April", "Prod4") : 500,
    ("April", "Prod5") : 200,
    ("April", "Prod6") : 0,
    ("April", "Prod7") : 100,
    ("May", "Prod1") : 0,
    ("May", "Prod2") : 100,
    ("May", "Prod3") : 500,
    ("May", "Prod4") : 100,
    ("May", "Prod5") : 1000,
    ("May", "Prod6") : 300,
    ("May", "Prod7") : 0,
    ("June", "Prod1") : 500,
    ("June", "Prod2") : 500,
    ("June", "Prod3") : 100,
    ("June", "Prod4") : 300,
    ("June", "Prod5") : 1100,
    ("June", "Prod6") : 500,
    ("June", "Prod7") : 60,
}

In [8]:
storeCost = 0.5
storeCapacity = 100
endStock = 50
hoursPerMonth = 2*8*24

In [9]:
model = Model('Factory Planning I')


Using license file /Users/peyman/gurobi.lic
Academic license - for non-commercial use only


In [10]:
manu = model.addVars(time_periods, products, name="Manu") # quantity manufactured
held = model.addVars(time_periods, products, ub=storeCapacity, name="Held") # quantity stored
sell = model.addVars(time_periods, products, ub=upper, name="Sell") # quantity sold

In [23]:
manu['January', 'Prod1']

<gurobi.Var Manu[January,Prod1] (value 500.0)>

In [11]:
# Initial Balance
model.addConstrs((manu[time_periods[0], product] == sell[time_periods[0], product] 
                  + held[time_periods[0], product] for product in products), name="Initial_Balance")

{'Prod1': <gurobi.Constr *Awaiting Model Update*>,
 'Prod2': <gurobi.Constr *Awaiting Model Update*>,
 'Prod3': <gurobi.Constr *Awaiting Model Update*>,
 'Prod4': <gurobi.Constr *Awaiting Model Update*>,
 'Prod5': <gurobi.Constr *Awaiting Model Update*>,
 'Prod6': <gurobi.Constr *Awaiting Model Update*>,
 'Prod7': <gurobi.Constr *Awaiting Model Update*>}

Interesting technique to get the last time index

In [12]:
#Balance
model.addConstrs((held[time_periods[time_periods.index(time_period) -1], product] + 
                manu[time_period, product] == sell[time_period, product] + held[time_period, product] 
                for product in products for time_period in time_periods 
                if time_period != time_periods[0]), name="Balance")

{('Prod1', 'February'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod1', 'March'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod1', 'April'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod1', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod1', 'June'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod2', 'February'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod2', 'March'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod2', 'April'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod2', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod2', 'June'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod3', 'February'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod3', 'March'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod3', 'April'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod3', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod3', 'June'): <gurobi.Constr *Awaiting Model Update*>,
 ('Prod4', 'February'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pr

In [13]:
#End store
model.addConstrs((held[time_periods[-1], product] == endStock for product in products),  name="End_Balance")

{'Prod1': <gurobi.Constr *Awaiting Model Update*>,
 'Prod2': <gurobi.Constr *Awaiting Model Update*>,
 'Prod3': <gurobi.Constr *Awaiting Model Update*>,
 'Prod4': <gurobi.Constr *Awaiting Model Update*>,
 'Prod5': <gurobi.Constr *Awaiting Model Update*>,
 'Prod6': <gurobi.Constr *Awaiting Model Update*>,
 'Prod7': <gurobi.Constr *Awaiting Model Update*>}

In [14]:
# Capacity
model.addConstrs((quicksum(time_table[machine][product] * manu[time_period, product] 
	for product in time_table[machine]) <= hoursPerMonth * (qMachine[machine] - 
	down[time_period, machine]) for machine in machines for time_period in time_periods 
	if (time_period, machine) in down), name = "Capacity")

{('grinder', 'January'): <gurobi.Constr *Awaiting Model Update*>,
 ('grinder', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('vertDrill', 'April'): <gurobi.Constr *Awaiting Model Update*>,
 ('vertDrill', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('horiDrill', 'February'): <gurobi.Constr *Awaiting Model Update*>,
 ('horiDrill', 'June'): <gurobi.Constr *Awaiting Model Update*>,
 ('borer', 'March'): <gurobi.Constr *Awaiting Model Update*>,
 ('planer', 'June'): <gurobi.Constr *Awaiting Model Update*>}

In [24]:
model.addConstrs((quicksum(time_table[machine][product] * manu[time_period, product] 
	for product in time_table[machine]) <= hoursPerMonth * qMachine[machine] 
    for machine in machines for time_period in time_periods 
    if (time_period, machine) not in down), name = "Capacity")

{('grinder', 'February'): <gurobi.Constr *Awaiting Model Update*>,
 ('grinder', 'March'): <gurobi.Constr *Awaiting Model Update*>,
 ('grinder', 'April'): <gurobi.Constr *Awaiting Model Update*>,
 ('grinder', 'June'): <gurobi.Constr *Awaiting Model Update*>,
 ('vertDrill', 'January'): <gurobi.Constr *Awaiting Model Update*>,
 ('vertDrill', 'February'): <gurobi.Constr *Awaiting Model Update*>,
 ('vertDrill', 'March'): <gurobi.Constr *Awaiting Model Update*>,
 ('vertDrill', 'June'): <gurobi.Constr *Awaiting Model Update*>,
 ('horiDrill', 'January'): <gurobi.Constr *Awaiting Model Update*>,
 ('horiDrill', 'March'): <gurobi.Constr *Awaiting Model Update*>,
 ('horiDrill', 'April'): <gurobi.Constr *Awaiting Model Update*>,
 ('horiDrill', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('borer', 'January'): <gurobi.Constr *Awaiting Model Update*>,
 ('borer', 'February'): <gurobi.Constr *Awaiting Model Update*>,
 ('borer', 'April'): <gurobi.Constr *Awaiting Model Update*>,
 ('borer', 'May'): 

In [25]:
# Objective
obj = quicksum(
	  profit_contribution[product] * sell[time_period, product] -  
	  storeCost * held[time_period, product]  
	  for time_period in time_periods for product in products)

In [26]:
model.setObjective(obj, GRB.MAXIMIZE)

In [27]:
model.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 101 rows, 126 columns and 376 nonzeros
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [6e+01, 1e+03]
  RHS range        [5e+01, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.3715179e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  9.371517857e+04


In [28]:
for v in model.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

Manu[January,Prod1] 500.000000
Manu[January,Prod2] 888.571429
Manu[January,Prod3] 382.500000
Manu[January,Prod4] 300.000000
Manu[January,Prod5] 800.000000
Manu[January,Prod6] 200.000000
Manu[February,Prod1] 700.000000
Manu[February,Prod2] 600.000000
Manu[February,Prod3] 117.500000
Manu[February,Prod5] 500.000000
Manu[February,Prod6] 300.000000
Manu[February,Prod7] 250.000000
Manu[March,Prod6] 400.000000
Manu[April,Prod1] 200.000000
Manu[April,Prod2] 300.000000
Manu[April,Prod3] 400.000000
Manu[April,Prod4] 500.000000
Manu[April,Prod5] 200.000000
Manu[April,Prod7] 100.000000
Manu[May,Prod2] 100.000000
Manu[May,Prod3] 600.000000
Manu[May,Prod4] 100.000000
Manu[May,Prod5] 1100.000000
Manu[May,Prod6] 300.000000
Manu[May,Prod7] 100.000000
Manu[June,Prod1] 550.000000
Manu[June,Prod2] 550.000000
Manu[June,Prod4] 350.000000
Manu[June,Prod6] 550.000000
Held[January,Prod3] 82.500000
Held[February,Prod1] 100.000000
Held[February,Prod2] 100.000000
Held[February,Prod5] 100.000000
Held[February,Prod